In [1]:
import pandas as pd
import plotly.express as px

c:\Users\pcost\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [12]:
athlete_Bio = pd.read_csv('Dataset/Olympic_Athlete_Bio.csv', sep=',')
athlete_Bio = athlete_Bio.replace(["na"], None)
athlete_Bio["height"] = athlete_Bio["height"].astype(float)

athlete_Event_Results = pd.read_csv('Dataset/Olympic_Athlete_Event_Results.csv', sep=',')
athlete_Event_Results = athlete_Event_Results.replace(["na"], None)

In [54]:
a_bio = {'athelete_id': athlete_Bio['athlete_id'], 'sex': athlete_Bio['sex']}
a_event = {'athelete_id': athlete_Event_Results['athlete_id'], 'medal': athlete_Event_Results['medal']}

df = pd.merge(pd.DataFrame(a_bio), pd.DataFrame(a_event), on='athelete_id')

count = pd.DataFrame(df.groupby(['sex']).count())
count = count['athelete_id']

df1 = df.dropna(subset=['medal'])

print(count)

perc = pd.DataFrame(df1.groupby(['sex', 'medal']).count()).reset_index()


perc.loc[perc["sex"] == "Male", "athelete_id"] = (perc['athelete_id']/count[1])*100
perc.loc[perc["sex"] == "Female", "athelete_id"] = (perc['athelete_id']/count[0])*100

perc.rename(columns={"athelete_id": "count"}, inplace=True)
print(perc)

fig = px.sunburst(perc, path=['medal', 'sex'], values='count', color_discrete_map={'Gold': '#D6AF36', 'Silver': '#A7A7AD', 'Bronze': '#A77044'})
fig.show()


sex
Female     89310
Male      225597
Name: athelete_id, dtype: int64
      sex   medal     count
0  Female  Bronze  5.040869
1  Female    Gold  5.002799
2  Female  Silver  4.969208
3    Male  Bronze  4.610877
4    Male    Gold  4.680470
5    Male  Silver  4.522223


In [67]:
a_bio = {'athelete_id': athlete_Bio['athlete_id'], 'sex': athlete_Bio['sex']}
a_event = {'athelete_id': athlete_Event_Results['athlete_id'], 'edition': athlete_Event_Results['edition']}
a_event['edition'] = a_event['edition'].str.split(' ')

a_event = {'athelete_id': a_event['athelete_id'], 'year': a_event['edition'].str[0], 'season': a_event['edition'].str[1]}

df = pd.merge(pd.DataFrame(a_bio), pd.DataFrame(a_event), on='athelete_id')

count = pd.DataFrame(df.groupby(['year', 'sex', 'season']).count()).reset_index()
count.rename(columns={"athelete_id": "count"}, inplace=True)

print(count)

summer = count[count['season'] == 'Summer']
winter = count[count['season'] == 'Winter']

fig = px.sunburst(summer, path=['year', 'sex'], values='count')
fig.show()

fig = px.sunburst(winter, path=['year', 'sex'], values='count')
fig.show()

     year     sex  season  count
0    1896  Female  Summer      2
1    1896    Male  Summer    617
2    1900  Female  Summer     53
3    1900    Male  Summer   3094
4    1904  Female  Summer     17
..    ...     ...     ...    ...
104  2018    Male  Winter   3036
105  2020  Female  Summer   7126
106  2020    Male  Summer   7660
107  2022  Female  Winter   2430
108  2022    Male  Winter   2958

[109 rows x 4 columns]
